<a href="https://colab.research.google.com/github/suprapraba2923/HumAInSubmission/blob/master/Taxonomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.TrainedModels import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
import pandas as pd
from collections import Counter
from sklearn.TrainedModel_selection import train_test_split
from nltk import word_tokenize
from collections import defaultdict


def CountFirstXWords(corpus, top_x, skip_top_n):
    count = defaultdict(lambda: 0)
    for c in corpus:
        for w in word_tokenize(c):
            count[w] += 1
    count_tuples = sorted([(w, c) for w, c in count.items()], key=lambda x: x[1], reverse=True)
    return [i[0] for i in count_tuples[skip_top_n: skip_top_n + top_x]]


def VectorizeFirstWords(corpus, top_x):
    topx_dict = {top_x[i]: i for i in range(len(top_x))}

    return [
               [topx_dict[w] for w in word_tokenize(s) if w in topx_dict]
               for s in corpus
           ], topx_dict


def FilterFirst(corpus, n_top, skip_n_top=0):
    top_x = CountFirstXWords(corpus, n_top, skip_n_top)
    return VectorizeFirstWords(corpus, top_x)


df = pd.read_csv('SmallTrain.csv')

CompleteList = df['tags'].tolist()
counter = Counter(df['tags'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['tags'].map(lambda x: x in top_10_varieties)]

description_list = df['post'].tolist()
mapped_list, word_list = FilterFirst(description_list, 2500, 10)
ListOfVarieties_o = [top_10_varieties[i] for i in df['tags'].tolist()]
ListOfVarieties = to_categorical(ListOfVarieties_o)

max_review_length = 150

mapped_list = sequence.pad_sequences(mapped_list, maxlen=max_review_length)
train_x, test_x, train_y, test_y = train_test_split(mapped_list, ListOfVarieties, test_size=0.3)

max_review_length = 150

embedding_vector_length = 64
TrainedModel = Sequential()

TrainedModel.add(Embedding(2500, embedding_vector_length, input_length=max_review_length))
TrainedModel.add(Conv1D(50, 5))
TrainedModel.add(Flatten())
TrainedModel.add(Dense(101, activation='relu'))
TrainedModel.add(Dense(max(ListOfVarieties_o) + 1, activation='softmax'))
TrainedModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
TrainedModel.fit(train_x, train_y, epochs=3, batch_size=65)

TrainedModel.save('TrainedModelSave.h5')

# json_file = TrainedModel.to_json()
# with open("TrainedModel.json", "w") as file:
#   file.write(json_file)# serialize weights to HDF5
# TrainedModel.save_weights(h5_file)


# input_list = {"Hello, python, Image processing","Hello, python, Image processing"}
# print(TrainedModel.predict(test_x))

y_score = TrainedModel.predict(test_x)

idxs = np.argsort(y_score[0][0])[::-1][:2]

print(idxs)

# y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]
# n_right = 0
# for i in range(len(y_score)):
#     if all(y_score[i][j] == test_y[i][j] for j in range(len(y_score[i]))):
#         n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 23.93%
